In [13]:
import requests as rq
from bs4 import BeautifulSoup as bs
from collections import OrderedDict as od
from threading import Thread
from queuelib import queue
from selenium import webdriver
from datetime import datetime
import time
import random
import json
import csv
import traceback as tb
import re

In [14]:
HOST = 'https://www.famicloud.com.tw'

In [15]:
index_url = HOST

# 上層爬蟲

In [31]:
def famicloud_crawler():
    res_index = rq.get(index_url)
    soup_index = bs(res_index.text, 'lxml')
    soup_li = soup_index.li
    
    
    for i in range (1 ,1000):
        try:
            small_branch = soup_li.select('a')[i].text
            pre_small_branch_url = soup_li.select('a')[i]['href']
            small_branch_url = "{}{}".format(index_url, pre_small_branch_url)
            
            print(small_branch_url)
            #交給中層爬蟲
            #pass
            small_branch_crawler(small_branch_url)

        except KeyError:       
            sub_branch = soup_li.select('a')[i].text
            print(sub_branch)

        except IndexError:
            break   

    print("[INFO] famicloud_crawler complete!")

# 中層爬蟲

In [28]:
def small_branch_crawler(small_branch_url):
    # 擷取 JavaScript 渲染的網頁原始碼，先指定 PhantomJs 安裝位置
    driver = webdriver.PhantomJS(executable_path='C:/Users/nick800608/phantomjs-2.1.1-windows/bin/phantomjs')
     # 輸入範例網址，交給瀏覽器 
    driver.get(small_branch_url)
    # 取得網頁原始碼 (含 JavaScript)
    pageSource = driver.page_source    
    # print(pageSource)
    soup = bs(pageSource, 'lxml')
    
    a_href_list = soup.select('#SmallMedium > a')
    for a_href in a_href_list:
        if a_href['href'] != 'javascript:void(0)':
            pre_product_url = a_href['href']
            product_url = "{}{}".format(HOST, pre_product_url)
            print(product_url)
            product_page_crawler(product_url)
            

# 下層爬蟲

In [29]:
def product_page_crawler(product_url):
    res = rq.get(product_url)
    soup = bs(res.text, 'lxml')
    
    product_dict = od()
    food_info_dict = od()  
    
    food_name = ''
    
    #找到最後一個有strong標籤的div，就會是內容
    div_index_needed = 0
    for div_index in range(1, 70):
        if hasattr(soup.select("div")[div_index].strong, 'text'):
            if div_index > div_index_needed:
                div_index_needed = div_index
            #print(div_index_needed)
    #print(div_index_needed)
    
    try:
        soup.select("div")[div_index_needed].strong.text
        content = soup.select("div")[div_index_needed]
        new_content = content
        
        while(new_content.strong != None):
            
            try:
                food_name += new_content.strong.text
                new_content.strong.extract()
                
            except AttributeError:
                break
                
        food_info_dict["food_name"] = food_name
        #print(food_info_dict)
        
    #------以下應該都沒用了-----
    except AttributeError:
        soup.select("div")[54].strong.text
        content = soup.select("div")[54]
        new_content = content
        
        while(new_content.strong != None):
            
            try:   
                food_name += new_content.strong.text
                new_content.strong.extract()
                
            except AttributeError:
                break
                
        food_info_dict["food_name"] = food_name
    #------以上應該都沒用了-----    
    
    except:
        print("[***ERROR***]can not find food_name!")
        
    while(True):
        
        try:
            new_content.br.replace_with(' ')
            
        except AttributeError:
            break
   
    food_info_and_other = new_content.text.strip().split() 
    
    #original_price
    food_info_dict["original_price"] = soup.select(".BOriginalPrice")[0].text
    #discount_price
    food_info_dict["discount_price"] = soup.select(".OnSalePrice > b")[0].text
    #stored_method
    food_info_dict["stored_method"] = soup.select(".BProductLabel")[0].img['alt']
    #category
    category_list = []
    a_hrefs = soup.select('#map-list')[0]
    branch_levels = a_hrefs.select('a')
    for branch in branch_levels:
        if(branch.text != '首頁'):
            category_list.append(branch.text)
    food_info_dict["category"] = category_list
    #weight
    food_info_dict["weight"] = food_info_and_other[1]
    #expired_date
    food_info_dict["expired_date"] = food_info_and_other[7]
    #country_of_origin
    food_info_dict["country_of_origin"] = food_info_and_other[5]
    
    #把food_info字典放入product_dict字典
    product_dict["food_info"] = food_info_dict
    
    #steps(未完成)
    #product_dict["steps"] = null
    
    #product_img_url
    product_dict["product_img_url"] = soup.select(".product_img_box")[0].img['src']
    #product_characteristic_img_url
    
    try:
        #不在P標籤內
        product_dict["product_characteristic_img_url"] = soup.select('div > img')[1]['src']
        #rint(product_dict)
        
    except IndexError:
        #在P標籤內
        product_dict["product_characteristic_img_url"] = soup.select("div > p > img")[0]['src']
    
    #web_url
    product_dict["web_url"] = product_url
    
    print(product_dict)

In [ ]:
famicloud_crawler()